In [1]:
library(Seurat)
library(BisqueRNA)
library(fastSave)
library(pbmcapply)


The legacy packages maptools, rgdal, and rgeos, underpinning the sp package,
which was just loaded, were retired in October 2023.
Please refer to R-spatial evolution reports for details, especially
https://r-spatial.org/r/2023/05/15/evolution4.html.
It may be desirable to make the sf package available;
package maintainers should consider adding sf to Suggests:.

Attaching SeuratObject

Loading required package: parallel



In [2]:
ref_seurat=readRDS.lbzip2(file='~/ReCIDE/benchmark测试/mult_reference_PBMC/ref_data/ref_all.rdsFS',n.cores = 200)
EXP<- readRDS("~/ReCIDE/benchmark测试/mult_reference_PBMC/EXP_and_KEY/EXP_PBMC.rds")



In [3]:
##构建测试集
bulk.mtx<-as.data.frame(EXP[[1]])
for (i in 2:length(EXP)) {
  bulk.mtx2<-as.data.frame(EXP[[i]])
  bulk.mtx<-cbind(bulk.mtx,bulk.mtx2)
  
}
bulk.mtx<-as.matrix(bulk.mtx)
colnames(bulk.mtx)<-names(EXP)
bulk.eset <- Biobase::ExpressionSet(assayData = bulk.mtx)

In [5]:
###########################
kk<-as.data.frame(ref_seurat@assays[["RNA"]]@counts)
kksum<-as.data.frame(apply(kk, 2, sum))
all(kksum>0)
kksum<-subset(kksum,kksum[,1]>0,)
kksum<-as.data.frame(kksum)
ref_seurat<-ref_seurat[,row.names(kksum)]
rm(kksum)
rm(kk)

gc()

sample.ids <- colnames(ref_seurat)
individual.labels<-ref_seurat@meta.data['batch']
cell.type.labels<-ref_seurat@meta.data['cell_label']

sc.pheno <- data.frame(check.names=F, check.rows=F,
                       stringsAsFactors=F,
                       row.names=sample.ids,
                       SubjectName=individual.labels,
                       cellType=cell.type.labels)

sc.meta <- data.frame(labelDescription=c("SubjectName",
                                         "cellType"),
                      row.names=c("SubjectName",
                                  "cellType"))
sc.pdata <- new("AnnotatedDataFrame",
                data=sc.pheno,
                varMetadata=sc.meta)

sc.eset <- Biobase::ExpressionSet(assayData=as.matrix(ref_seurat@assays$RNA@counts),
                                  phenoData=sc.pdata)
colnames(sc.eset@phenoData@data)<-c("SubjectName","cellType")
bisque_output <- BisqueRNA::ReferenceBasedDecomposition(bulk.eset, sc.eset, markers=NULL, use.overlap=FALSE)

# saveRDS(res,file = '~/SWORD/多参考集和单参考集比较测试/除SWORD外其他方法/bisque/TNBC_QEGAD_ref161529.rds')



Warning message in asMethod(object):
“sparse->dense coercion: allocating vector of size 1.1 GiB”


[1] FALSE

,used,(Mb),gc trigger,(Mb),max used,(Mb)
Ncells,3800316,203.0,6658877,355.7,6658877,355.7
Vcells,230227192,1756.5,729682682,5567.1,667306323,5091.2


Warning message in asMethod(object):
“sparse->dense coercion: allocating vector of size 1.1 GiB”
Decomposing into 11 cell types.

Using 592 genes in both bulk and single-cell expression.

Converting single-cell counts to CPM and filtering zero variance genes.

Filtered 0 zero variance genes.

Converting bulk counts to CPM and filtering unexpressed genes.

Filtered 0 unexpressed genes.

Generating single-cell based reference from 258196 cells.


Inferring bulk transformation from single-cell alone.

Applying transformation to bulk samples and decomposing.



In [6]:
saveRDS(bisque_output,file='~/ReCIDE/benchmark测试/mult_reference_PBMC/bisque/bisque_tutorial/bisque_output.rds')

In [1]:
bisque_output=readRDS('~/ReCIDE/benchmark测试/mult_reference_PBMC/bisque/bisque_tutorial/bisque_output.rds')

In [2]:
prd_df <-as.data.frame(bisque_output[["bulk.props"]])
prd_df=prd_df[sort(row.names(prd_df)),sort(colnames(prd_df))]

In [3]:
saveRDS(prd_df,file='~/ReCIDE/benchmark测试/mult_reference_PBMC/bisque/bisque_tutorial/bisque_prd_df.rds')

In [7]:
key_df<- readRDS("~/ReCIDE/benchmark测试/mult_reference_PBMC/EXP_and_KEY/key_PBMC.rds")
key_df=key_df[sort(row.names(key_df)),colnames(prd_df)]


In [9]:
RMSE_vec=c()
for(i in 1:length(key_df)){
  RMSE_vec[i]=ModelMetrics::rmse(key_df[,i],prd_df[,i])
  # RMSE_vec[i]=cor(key_df[,i],prd_df[,i])
  
}
median(RMSE_vec)

[1] 0.1648558

In [11]:
saveRDS(prd_df,file='~/ReCIDE/benchmark测试/mult_reference_PBMC/bisque/bisque_tutorial/bisque_prd_df.rds')

In [12]:
key_df

,Sample-100,Sample-17,Sample-19,Sample-22,Sample-27,Sample-33,Sample-37,Sample-43,Sample-45,Sample-46,Sample-47,Sample-50,Sample-52,Sample-58,Sample-66,Sample-82,Sample-90,Sample-96
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
Ast,0.101420765,0.066104820,0.103968514,0.05344045,0.146641691,0.06955871,0.07795560,0.058117890,0.030951876,0.026948711,0.13794244,0.10768072,0.049742710,0.08816210,0.06524773,0.05778302,0.113323124,0.060165511
Exc,0.121092896,0.062154332,0.185962611,0.04889976,0.074926412,0.18324607,0.25425916,0.143743537,0.058299767,0.111851637,0.10651671,0.10993976,0.008919383,0.01279772,0.05163992,0.06603774,0.030627871,0.143592038
Inh,0.092677596,0.050566237,0.102656609,0.01851205,0.180626171,0.16379955,0.17010842,0.094519131,0.050243799,0.078817734,0.18458485,0.14043675,0.003087479,0.02097405,0.09420796,0.17256289,0.070826953,0.087899799
Mic,0.031475410,0.067158283,0.088225648,0.06845966,0.101418250,0.10134630,0.07124419,0.057497415,0.034131863,0.064618951,0.08534568,0.07793675,0.060034305,0.05723427,0.08967202,0.07665094,0.070061256,0.064415120
Oli,0.612896175,0.720042139,0.457527058,0.77401327,0.416376773,0.40800299,0.35828601,0.607445708,0.797964808,0.681541582,0.43565994,0.47929217,0.850085763,0.76857448,0.64340544,0.54048742,0.630934150,0.599865802
Opc,0.035846995,0.032920727,0.052804198,0.03143556,0.074926412,0.05160808,0.05807950,0.035780765,0.026711893,0.032454361,0.03440291,0.06438253,0.021612350,0.03163882,0.04954641,0.07507862,0.080015314,0.040035786
Vas,0.004590164,0.001053463,0.008855362,0.00523926,0.005084292,0.02243829,0.01006711,0.002895553,0.001695993,0.003767024,0.01554747,0.02033133,0.006518010,0.02061856,0.00628053,0.01139937,0.004211332,0.004025945


In [13]:
prd_df

,Sample-100,Sample-17,Sample-19,Sample-22,Sample-27,Sample-33,Sample-37,Sample-43,Sample-45,Sample-46,Sample-47,Sample-50,Sample-52,Sample-58,Sample-66,Sample-82,Sample-90,Sample-96
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
Ast,0.1019724161,0.05397435,0.08147405,0.06489210,0.129981812,0.06983153,0.04097529,0.04107560,0.02060750,0.011778913,0.06740937,0.114738571,0.02992109,0.06645950,0.08835356,0.0645144744,0.12096802,0.052195381
Exc,0.4781085819,0.31011830,0.50274168,0.07525710,0.295042459,0.52358188,0.72812214,0.59428008,0.34127345,0.555610974,0.55632109,0.452072738,0.06342149,0.10724661,0.35557462,0.2723616235,0.18777089,0.459021936
Inh,0.1516092345,0.14584745,0.14538159,0.08693398,0.252239544,0.15284373,0.14169252,0.12005850,0.14607580,0.126778569,0.25069301,0.221879235,0.05575312,0.09662061,0.21274068,0.1801526177,0.10827043,0.120982357
Mic,0.0041412176,0.04654077,0.04159317,0.07576407,0.058223088,0.02298751,0.00000000,0.01531999,0.02724900,0.014759726,0.01290725,0.002964027,0.05292988,0.04576176,0.04203837,0.0591538653,0.05113585,0.026037909
Oli,0.2469757759,0.37476105,0.17917654,0.62681333,0.187688983,0.15480276,0.05130252,0.19467219,0.41190787,0.250153003,0.08711823,0.143663634,0.73413244,0.62021507,0.26387340,0.3091606089,0.40458833,0.296800754
Opc,0.0165310276,0.03484840,0.04963297,0.04826238,0.074481147,0.07595259,0.03790753,0.02335874,0.02950492,0.035418186,0.02555105,0.064681794,0.03051434,0.04028946,0.03741937,0.1136619769,0.12726647,0.038412055
Vas,0.0006617464,0.03390967,0.00000000,0.02207704,0.002342966,0.00000000,0.00000000,0.01123490,0.02338148,0.005500627,0.00000000,0.000000000,0.03332763,0.02340699,0.00000000,0.0009948334,0.00000000,0.006549608
